# PEN: Process Estimator neural Network for root cause analysis using graph convolution

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import sparse
import gcChebyshev

import ipywidgets

## Loading
We need to load all deviation fields, the values of the vertices, in a single variable.

In [ ]:
loadPath = '../Variationsanalyse/data/sto_2/run__0{:04d}/m_1/assembly_6.csv'
numberOfTests = 793
n = 188559
df = np.zeros((numberOfTests, n, 3), dtype=np.float32)
for testId in range(0, numberOfTests):
    df_temp = pd.read_csv(loadPath.format(testId+1), sep=' ', header=0, dtype=np.float32)
    df[testId,:] = np.reshape(df_temp.values, (-1,3))
    
print(df.shape)

In addition, we need the adjacency matrix.

In [ ]:
A_sparse = sparse.load_npz('../Variationsanalyse/data/sto_2/adjacencyList.npz')


## Load the labels

In [ ]:
labels = np.loadtxt('../Variationsanalyse/data/sto_2/run_matrix_Stoch2_updated_ESP.csv', delimiter=',', dtype=np.float32)

print(labels.shape)

# Model definition

In [ ]:
model = tf.keras.models.Sequential()

num_filters = 25
filter_size = 15
max_frequency = 30
smoothing_cut = max_frequency * 0.9
smoothing_width = max_frequency * 0.1

dense1 = 2048
dense1_activation = 'relu'
dense1_bias = False

dense2 = 6
dense2_activation = None
dense2_bias = True

model.add(gcChebyshev.gcChebyshev(A=A_sparse, num_filters=num_filters, filter_size=filter_size, smoothing_cut=smoothing_cut, smoothing_width=smoothing_width, max_frequency=max_frequency, input_shape=(df.shape[1], df.shape[2])))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(dense1, activation=dense1_activation, use_bias=dense1_bias))
model.add(tf.keras.layers.Dense(dense2, activation=dense2_activation, use_bias=dense2_bias))

model.summary()

# Training

In [ ]:
batchSize = 12
    
optimizer = tf.optimizers.Adagrad(
    learning_rate=0.05
)

In [ ]:
loss = 'mae'
epochs = 300

model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
hist = model.fit(df, labels, batch_size=batchSize, epochs=epochs, verbose=1)

In [ ]:
plt.figure(figsize=(18,6))
plt.subplot(2,1,1)
plt.plot(hist.history['loss'][2:], label='train')
plt.yscale('log')
#plt.plot(hist.history['val_loss'][2:], label='test')
plt.legend()
plt.subplot(2,1,2)
plt.plot(hist.history['accuracy'], label='train')
#plt.plot(hist.history['val_accuracy'], label='test')
plt.legend()
plt.show()

# Inspecting the results

In [ ]:
@ipywidgets.interact_manual(idx=(0,len(labels), 1))
def showReconstructed(idx=0):
    reconstructed = model(df[idx:idx+1])[0,:]
    
    print(np.abs(labels[idx] - reconstructed))
    
    f = plt.figure(figsize=(18,5))
    
    plt.plot(labels[idx], 'k', label='input')
    plt.plot(reconstructed, 'b', label='reconstructed')
    plt.plot(np.abs(labels[idx] - reconstructed), 'r', label='error')
    plt.legend()
    plt.xticks(range(6),labels=['ESP_L', 'ESP_R', 'SB_L', 'SN_X', 'SN_Y', 'SB_R'])
    
    plt.show()

# Store the model

In [ ]:
model.save('gcChebyshev.h5')